In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py -O minsearch.py

--2024-07-01 03:23:59--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-01 03:23:59 (22.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import minsearch

In [4]:
import requests

In [9]:
url = "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json"

In [10]:
response = requests.get(url)

In [11]:
file_content = response.json()

In [15]:
documents = []
for course_dict in file_content:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [16]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [17]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [18]:
q = 'the course has already started, can I still enroll?'

In [19]:
index.fit(documents)

In [47]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [48]:
search_results = search(q)

In [22]:
from openai import OpenAI

In [24]:
client = OpenAI()

In [53]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

context = ""

for doc in search_results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = prompt_template.format(question=q, context=context).strip()


In [54]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'Yes, you can still enroll in the course even after it has started. You are eligible to submit the homework assignments, but be aware of the deadlines for turning in the final projects. Make sure not to leave everything until the last minute.'

In [37]:
build_prompt(query)

TypeError: build_prompt() missing 1 required positional argument: 'search_results'